# Лабораторная работа №2 по дисциплине "Математические методы цифровой обработки сигналов"

# Тема: Алгоритм Герцеля

#### Выполнил магистрант гр. 9385 Семенов Александр.
#### Вариант 7

## Постановка задачи
**Входные данные:** 
1) Аудио сигнал $x$ с частотой дискретизации $F=44100 Гц, 16 Bit, Mono, PCM wave file$.

2) В сигнале $x$ с помощью DTMF кодирования прописана последовательность различных цифр (и управляющих клавиш) телефона.

3) Длина каждого элемента звуковой последовательности $200 mc$, паузы между ними тоже по $200 mc$.

**Задача:** С помощью алгоритм Герцеля декодировать сигнал и восстановить исходную последовательность.

В таблице приведены DTMF тоны различных цифр (и управляющих клавиш) телефона и соответствующих им пар частот:


|Группа частот (Гц)|1209|1336|1477|1633|
|:-:|:-:|:-:|:-:|:-:|
|697|1|2|3|A|
|770|4|5|6|B|
|852|7|8|9|C|
|941|*|0|#|D|

## Выполнение работы

### 1. Инициализация данных, чтение исходного файла

In [75]:
import math
import librosa

toneTable = {
    697: {1209: '1', 1336: '2', 1477: '3', 1633: 'A'},
    770: {1209: '4', 1336: '5', 1477: '6', 1633: 'B'},
    852: {1209: '7', 1336: '8', 1477: '9', 1633: 'C'},
    941: {1209: '*', 1336: '0', 1477: '#', 1633: 'D'},
}

rate = 44100
timeInterval = 200
N = rate * timeInterval // 1000

x, _ = librosa.load("DTMF_Lab7.wav", sr=rate)

### 2. Реализация алгоритма

In [76]:
def hertsell(freq, frames, rate):
    g1 = g2 = float(0)
    for frame in frames:
        g1_new = frame + 2*math.cos(2*math.pi*freq/rate)*g1 - g2
        g2_new = g1
        g1, g2 = g1_new, g2_new
        
    return math.sqrt(g2**2 + g1**2 - 2*math.cos(2*math.pi*freq/rate)*g1*g2)

result = []

for i in range(0, 2 * len(x) // (3 * N)):
    frames = x[3 * i * N // 2 : 3 * i * N // 2 + N - 1][::-1]
    frequencies = sorted(toneTable.keys()) + sorted(toneTable[697].keys())
    values = {freq: hertsell(freq, frames, rate) for freq in frequencies}
    
    toneFreqs = [key for key in values if values[key] >= 500]
    result.append(toneTable[min(toneFreqs)][max(toneFreqs)])

### 3. Результат

In [77]:
print("".join(result))

9#0*1A2B3C4D5*6*


Ответ (декодированная последовательность кодов): 
## _9#0\*1A2B3C4D5\*6\*_